# The position of the app in itunes 

В процессе работы мы часто пользуемся разными источниками данных, и это не всегда готовые базы данных. Иногда данные нужно добывать с помощью API. И, допустим, нам интересно, на каких позициях наше мобильное приложение или приложение конкурентов. Чаще всего приложения ищутся по словам в их названии (не обязательно на первом месте). Для того, чтобы узнать конкретную позицию по конкретным словам, можно воспользоваться API iTunes.

In [1]:
from datetime import datetime  # для определения времени запроса
import requests  # для запроса к сайту itunes.apple.com
import os.path  # для проверки существования файла базы данных
import sqlite3  # для работы с таблицей
import pandas as pd  # для просмотра таблицы
import json

Пусть id - appleid приложения, которое мы хотим исследовать (например, 860011430). У этого приложения есть название из N слов (в данном случае Ghost Lens AR Fun MovieMaker). Я написал python-скрипт, который принимает на вход id и заносит в sqlite3 базу данных itunes.db в таблицу itunes (id, word, pos, date) N строк; в каждой строке написано id приложения, запрашиваемое слово из названия, позиция приложения id в
выдаче по этому слову и дата запуска скрипта.

In [2]:
def write_data(appleid):
    apps_id = 'https://itunes.apple.com/lookup?id={id}'.format(id=str(appleid))
    response_id = requests.get(apps_id)  # Создание запроса GET
    
    if response_id.status_code != 200:  # Проверка запроса на статус состояния 200
        return print('\nProblems with access to the server! Request error: {}\n'.format(response_id.status_code))
    
    json_response_id = response_id.json()
    n = response_id.json()['resultCount']  # Колличество выданных по запросу результатов
    
    if n == 0:  # Если n = 0, то по введённому id приложения не существует
        return print('\nFor this id, the application does not exist\n')
    
    name = list(json_response_id['results'])[0]['trackName']  # Название приложения в AppStore
    
    print('\nApplication Name:    ' + name + '\n')  # Вывод на экран этого названия 
    
    
    params = {
        'term' : '',
        'media' : 'software',  # рассматриваем только приложения
        'entity' : 'software',
        'limit' : 200  # максимум 200 результатов, согласно документации
    }
    url = 'https://itunes.apple.com/search?'
    
    words = name.split()  # слова по которым будет идти поиск
    data_itunes = list()
    date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    for word in words:
        params['term'] = word
        res = requests.get(url, params)
        names = [track['trackName'] for track in json.loads(res.text)['results']]
        if name in names:
            position = names.index(name) + 1
        else:
            position = 'Null'  # Если по запрашивающему из названия слову ничего не найдено
        data_itunes.append({'id': appleid, 'word': word, 'pos': position, 'date': date})
        
    con = sqlite3.connect('itunes.db')
    cur = con.cursor()
    for elem in data_itunes:
        cur.execute('INSERT INTO itunes VALUES (?, ?, ?,?)', (elem['id'], elem['word'], elem['pos'], elem['date']))
    con.commit()
    cur.close()
    con.close()
    
    
# Если база данных ещё не создана, то создадим
if os.path.exists('itunes.db') is not True:
    con = sqlite3.connect('itunes.db')
    cur = con.cursor()
    # Создание таблицы, соответствующей условию
    cur.execute("""CREATE TABLE itunes(
        id INT,
        word TEXT,
        pos INT,
        date DATE);
    """)
    con.commit()
    cur.close()
    con.close()

write_data(appleid = 860011430)  # сюда вводится номер id, например: 860011430



Application Name:    Ghost Lens AR Fun Movie Maker



In [3]:
# Теперь посмотрим на результат того, что получилось
db = sqlite3.connect('itunes.db')
df = pd.read_sql_query('SELECT * FROM itunes', db)
db.close()
df.tail(10)

,id,word,pos,date
8,860011430,AR,Null,2022-08-10 19:03:12
9,860011430,Fun,Null,2022-08-10 19:03:12
10,860011430,Movie,37,2022-08-10 19:03:12
11,860011430,Maker,88,2022-08-10 19:03:12
12,860011430,Ghost,10,2022-08-10 19:07:17
13,860011430,Lens,8,2022-08-10 19:07:17
14,860011430,AR,Null,2022-08-10 19:07:17
15,860011430,Fun,Null,2022-08-10 19:07:17
16,860011430,Movie,37,2022-08-10 19:07:17
17,860011430,Maker,88,2022-08-10 19:07:17
